In [204]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyodbc
import numpy as np

In [284]:
# SQL servers credentials
live_samo = 'Driver={SQL Server};''Server=pck-sam-sql-04;''Database=TUI;''Trusted_Connection=yes;'
archive_samo = 'Driver={SQL Server};''Server=pvm-sam-sql-05.tui.local;''Database=TUI;''Trusted_Connection=yes;'
analytics = 'Driver={SQL Server};''Server=pvm-bi-sql-01;''Database=TUIOlap;''Trusted_Connection=yes;'

# function to pull data from sql table
def sql_pull(cred, sql_query):
    conn = pyodbc.connect(cred)
    cursor = conn.cursor()
    result = pd.read_sql_query(sql_query,conn)

    cursor.close()
    del cursor
    conn.close()
    
    return result

def samo_data():    
    return sql_pull(live_samo, ' \
                    SELECT DATEPART(MONTH, claim.datebeg) AS \'Checkin_Month\', \
                    claim.datebeg AS \'Checkin\', \
                           hotel.lname AS \'Hotel\', \
                           region.lname AS \'Hotel_region\', \
                           pax.adult + pax.child AS \'Pax\', \
                           claim.nights AS \'Room_night\', \
                           partner.plname AS \'Owner\', \
                           tour.lname AS \'Tour\' \
                    FROM claim \
                    LEFT JOIN [order] res \
                        ON claim.inc = res.claim \
                    LEFT JOIN hotel \
                        ON res.hotel = hotel.inc \
                    LEFT JOIN v_ClaimPax pax \
                        ON claim.inc = pax.inc \
                    LEFT JOIN tour \
                        ON claim.tour = tour.inc \
                    LEFT JOIN state \
                        ON tour.state = state.inc \
                    LEFT JOIN PartnerTownRegion AS partner \
                        ON claim.owner = partner.pinc \
                    LEFT JOIN town \
                        ON hotel.town = town.inc \
                    LEFT JOIN region \
                        ON town.region = region.inc \
                    WHERE res.hotel > 0 \
                          AND state.lname = \'UAE\' \
                          AND claim.datebeg >= \'2020-09-01 00:00:00\' \
                          AND hotel.lname IS NOT NULL')

In [285]:
data = samo_data()

filter_data = data[(data.Checkin >= '2020-09-01 00:00:00') & 
                   (data.Checkin <= '2021-05-31 23:59:59')]

In [242]:
# 3.
print('2021 PAX TOTAL by owner:')
third_report = data[(data.Hotel_region == 'Ras Al Khaimah') & 
                    (data.Checkin >= '01.01.2021 00:00:00') &
                    (data.Checkin <= '31.12.2021 23:59:59')] \
                                                  .groupby('Owner').sum().Pax \
                                                  .reset_index() \
                                                  .sort_values('Pax', ascending = False)

third_report.loc['TOTAL'] = third_report.sum()
third_report.loc[third_report.index == 'TOTAL', 'Owner'] = ' '
third_report.Pax = third_report.Pax.astype(int)

third_report = third_report.style.hide_index()
third_report

2021 PAX TOTAL by owner:


Owner,Pax
TUI RU,2473
Itour (Алматы),1420
TUI UA,278
TUI Uzbekistan (iTour),58
,4229


In [296]:
# 2a
for owner in filter_data.Owner.unique():
    print(owner, 'RKT Hotels,  sep 20 - may 21:')

    hotel_report = filter_data[(filter_data.Hotel_region == 'Ras Al Khaimah') &
                               (filter_data.Owner == owner)]  \
                        .groupby(['Hotel_region', 'Hotel']).sum()[['Pax', 'Room_night']] \
                        .sort_values('Pax', ascending = False)


    hotel_report['Share'] = hotel_report.Pax / hotel_report.Pax.sum() * 100
    hotel_report[['Pax', 'Room_night']] = hotel_report[['Pax', 'Room_night']].astype(int)
    #hotel_report.loc['TOTAL'] = hotel_report.sum()

    display(hotel_report)

Itour (Алматы) RKT Hotels,  sep 20 - may 21:


Pax  \
Hotel_region   Hotel                                                   
Ras Al Khaimah Rixos Bab Al Bahr                                2126   
               DoubleTree By Hilton Resort & SPA Marjan Island   490   
               Hilton Ras Al Khaimah Resort & Spa                 73   
               BM Beach Resort                                    54   
               The Village at The Cove Rotana Resort              45   
               BM Beach Hotel                                     41   
               Marjan Island Resort & SPA                         26   
               The Cove Rotana Resort Ras Al Khaimah              24   
               Waldorf Astoria Ras Al Khaimah                     21   
               Doubletree By Hilton Ras Al Khaimah                 4   

                                                                Room_night  \
Hotel_region   Hotel                                                         
Ras Al Khaimah Rixos Bab Al Bahr                                      4849   
               DoubleTree By Hilton Resort & SPA Marjan Island        1331   
               Hilton Ras Al Khaimah Resort & Spa                      230   
               BM Beach Resort                                         181   
               The Village at The Cove Rotana Resort                   101   
               BM Beach Hotel                                          151   
               Marjan Island Resort & SPA                               80   
               The Cove Rotana Resort Ras Al Khaimah                    67   
               Waldorf Astoria Ras Al Khaimah                           61   
               Doubletree By Hilton Ras Al Khaimah                       9   

                                                                    Share  
Hotel_region   Hotel                                                       
Ras Al Khaimah Rixos Bab Al Bahr                                73.209366  
               DoubleTree By Hilton Resort & SPA Marjan Island  16.873278  
               Hilton Ras Al Khaimah Resort & Spa                2.513774  
               BM Beach Resort                                   1.859504  
               The Village at The Cove Rotana Resort             1.549587  
               BM Beach Hotel                                    1.411846  
               Marjan Island Resort & SPA                        0.895317  
               The Cove Rotana Resort Ras Al Khaimah             0.826446  
               Waldorf Astoria Ras Al Khaimah                    0.723140  
               Doubletree By Hilton Ras Al Khaimah               0.137741

TUI RU RKT Hotels,  sep 20 - may 21:


Pax  \
Hotel_region   Hotel                                                     
Ras Al Khaimah Rixos Bab Al Bahr                                  1005   
               DoubleTree By Hilton Resort & SPA Marjan Island     821   
               Hilton Ras Al Khaimah Resort & Spa                  414   
               The Village at The Cove Rotana Resort               204   
               BM Beach Resort                                     124   
               Marjan Island Resort & SPA                          118   
               The Cove Rotana Resort Ras Al Khaimah               116   
               Al Hamra Residence                                   51   
               BM Beach Hotel                                       45   
               Waldorf Astoria Ras Al Khaimah                       45   
               Al Hamra Village Golf & Beach Resort                 43   
               Doubletree By Hilton Ras Al Khaimah                  19   
               Citymax Hotel Ras Al Khaimah                          8   
               Hilton Al Hamra Beach & Golf Resort                   4   
               Jannah Hotel Apartments and Villas Ras Al Khaimah     1   

                                                                  Room_night  \
Hotel_region   Hotel                                                           
Ras Al Khaimah Rixos Bab Al Bahr                                        2774   
               DoubleTree By Hilton Resort & SPA Marjan Island          2750   
               Hilton Ras Al Khaimah Resort & Spa                       1329   
               The Village at The Cove Rotana Resort                     608   
               BM Beach Resort                                           482   
               Marjan Island Resort & SPA                                527   
               The Cove Rotana Resort Ras Al Khaimah                     432   
               Al Hamra Residence                                        185   
               BM Beach Hotel                                            200   
               Waldorf Astoria Ras Al Khaimah                            151   
               Al Hamra Village Golf & Beach Resort                      164   
               Doubletree By Hilton Ras Al Khaimah                        62   
               Citymax Hotel Ras Al Khaimah                               47   
               Hilton Al Hamra Beach & Golf Resort                        16   
               Jannah Hotel Apartments and Villas Ras Al Khaimah           7   

                                                                      Share  
Hotel_region   Hotel                                                         
Ras Al Khaimah Rixos Bab Al Bahr                                  33.300199  
               DoubleTree By Hilton Resort & SPA Marjan Island    27.203446  
               Hilton Ras Al Khaimah Resort & Spa                 13.717694  
               The Village at The Cove Rotana Resort               6.759443  
               BM Beach Resort                                     4.108681  
               Marjan Island Resort & SPA                          3.909874  
               The Cove Rotana Resort Ras Al Khaimah               3.843605  
               Al Hamra Residence                                  1.689861  
               BM Beach Hotel                                      1.491054  
               Waldorf Astoria Ras Al Khaimah                      1.491054  
               Al Hamra Village Golf & Beach Resort                1.424785  
               Doubletree By Hilton Ras Al Khaimah                 0.629556  
               Citymax Hotel Ras Al Khaimah                        0.265076  
               Hilton Al Hamra Beach & Golf Resort                 0.132538  
               Jannah Hotel Apartments and Villas Ras Al Khaimah   0.033135

TUI UA RKT Hotels,  sep 20 - may 21:


...

,,Pax,Room_night,Share
Hotel_region,Hotel,,,


TUI Uzbekistan (iTour) RKT Hotels,  sep 20 - may 21:


Pax  \
Hotel_region   Hotel                                                  
Ras Al Khaimah Rixos Bab Al Bahr                                 55   
               DoubleTree By Hilton Resort & SPA Marjan Island    4   
               Marjan Island Resort & SPA                         3   

                                                                Room_night  \
Hotel_region   Hotel                                                         
Ras Al Khaimah Rixos Bab Al Bahr                                       150   
               DoubleTree By Hilton Resort & SPA Marjan Island           6   
               Marjan Island Resort & SPA                                7   

                                                                    Share  
Hotel_region   Hotel                                                       
Ras Al Khaimah Rixos Bab Al Bahr                                88.709677  
               DoubleTree By Hilton Resort & SPA Marjan Island   6.451613  
               Marjan Island Resort & SPA                        4.838710

TUI Baltics RKT Hotels,  sep 20 - may 21:


,,Pax,Room_night,Share
Hotel_region,Hotel,,,


In [299]:
# 2b
for owner in filter_data.Owner.unique():

    print(owner, 'UAE regions,  sep 20 - may 21:')
    region_report = filter_data[filter_data.Owner == owner] \
        .groupby('Hotel_region').sum()[['Pax', 'Room_night']] \
        .sort_values('Pax', ascending = False)

    region_report['Share'] = region_report.Pax / region_report.Pax.sum() * 100

    region_report[['Pax', 'Room_night']] = region_report[['Pax', 'Room_night']].astype(int)

    display(region_report)

Itour (Алматы) UAE regions,  sep 20 - may 21:


,Pax,Room_night,Share
Hotel_region,,,
Dubai,9456,25981,67.789806
Ras Al Khaimah,2904,7060,20.818697
Ajman,676,2094,4.846226
Sharjah,561,2234,4.021794
Fujairah,290,929,2.079002
Abu-Dhabi,59,178,0.422969
Umm Al Quwain,3,8,0.021507


TUI RU UAE regions,  sep 20 - may 21:


,Pax,Room_night,Share
Hotel_region,,,
Dubai,10881,38161,54.048281
Ras Al Khaimah,3018,9734,14.991059
Ajman,2995,12040,14.876813
Sharjah,1663,7139,8.260481
Fujairah,1322,5094,6.566660
Abu-Dhabi,223,735,1.107689
Umm Al Quwain,30,131,0.149016


TUI UA UAE regions,  sep 20 - may 21:


...

,Pax,Room_night,Share
Hotel_region,,,
Dubai,198,797,88.000000
Ajman,19,81,8.444444
Sharjah,8,33,3.555556


TUI Uzbekistan (iTour) UAE regions,  sep 20 - may 21:


,Pax,Room_night,Share
Hotel_region,,,
Dubai,1307,3755,93.490701
Ras Al Khaimah,62,163,4.434907
Sharjah,17,72,1.216023
Ajman,12,34,0.858369


TUI Baltics UAE regions,  sep 20 - may 21:


,Pax,Room_night,Share
Hotel_region,,,
Dubai,29,122,100.0
